In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [2]:
ruta = "C:/Users/9 ----- SIG/Documents/Análisis RM_051.xlsx"
df = pd.read_excel(ruta)

### CÓDIGO CON BUCLE

In [ ]:
# Crear el navegador
driver = webdriver.Chrome()

# Abrir la página de inicio de sesión
driver.get("https://hecaa.mineducacion.gov.co/hecaa-snies/content/admin/login/login.jsf")
time.sleep(1)

# Ingresar usuario y contraseña
usuario = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-user']")
usuario.send_keys("1826-admin")

contrasena = driver.find_element(By.CSS_SELECTOR, "input[id='loginForm:login-password']")
contrasena.send_keys("1826Uan9123")

# Clic en el botón de ingresar
boton_ingresar = driver.find_element(By.ID, "loginForm:submitLogin")
boton_ingresar.click()

# Ir a la sección MDM -> Novedad Poblacional
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_129"))).click()
time.sleep(1)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "menu_135"))).click()
time.sleep(1)

# Seleccionar tipo de documento UNA SOLA VEZ (Cédula de ciudadanía)
dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "filter:tipoDoc_label")))
dropdown.click()
time.sleep(0.5)
opcion = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//li[@data-label='Cédula de ciudadanía']")))
opcion.click()
time.sleep(0.5)

resultados = [] #informe al final del bucle

# Procesar cada registro del DataFrame
for i in range(len(df)):
    try:
        # Ingresar número de cédula
        cedula = str(df['NUM_DOCUMENTO'].iloc[i])
        input_cedula = driver.find_element(By.ID, "filter:numDoc")
        input_cedula.clear()
        input_cedula.send_keys(cedula)
        time.sleep(0.3)

        # Buscar
        boton_buscar = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Buscar']")))
        boton_buscar.click()
        time.sleep(1)

        # Clic en "Consultar"
        codigo_programa = str(df['CODIGO_PROGRAMA'].iloc[i])
        fila_codigo = WebDriverWait(driver, 4).until(
            EC.presence_of_element_located((By.XPATH, f"//td[contains(text(), '{codigo_programa}')]/parent::tr"))
        )
        boton_consultar = fila_codigo.find_element(By.TAG_NAME, "a")
        boton_consultar.click()
        time.sleep(0.3)

        # Clic en el lápiz (editar)
        burbuja_verde = WebDriverWait(driver, 4).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='small-circle color-border-green color-green' and text()='P']"))
        )
        lapiz = burbuja_verde.find_element(By.XPATH, "./parent::*//i[contains(@class, 'fa-pencil')]")
        lapiz.click()
        time.sleep(0.3)

        # Justificación: Error de reporte
        dropdown_justificacion = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.ID, "formEdit:justificacion_label")))
        dropdown_justificacion.click()
        time.sleep(0.3)
        opcion_error_reporte = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.ID, "formEdit:justificacion_2")))
        opcion_error_reporte.click()
        time.sleep(0.3)
        driver.find_element(By.TAG_NAME, "body").click()  # Forzar cierre del dropdown
        time.sleep(0.3)

        # Seleccionar período
        dropdown_periodo = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.ID, "formEdit:periodos_label")))
        dropdown_periodo.click()
        periodo_buscar = str(df['FORMATO'].iloc[i])
        opcion_periodo = WebDriverWait(driver, 4).until(
            EC.element_to_be_clickable((By.XPATH, f"//li[@data-label='{periodo_buscar}']"))
        )
        opcion_periodo.click()
        driver.find_element(By.TAG_NAME, "body").click()
        time.sleep(0.3)

        # Esperar a que desaparezca el overlay de carga, si aparece
        try:
            WebDriverWait(driver, 10).until(
                EC.invisibility_of_element_located((By.ID, "loadingDialogId_modal"))
            )
        except:
            pass
        
        # Ahora sí, hacer clic en "Mover Primer Curso"
        boton_mover_primer_curso = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='Mover Primer Curso']"))
        )
        boton_mover_primer_curso.click()
        time.sleep(0.3)

        # Clic en "Volver"
        boton_volver = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'ui-button-text') and contains(text(), 'Volver')]"))
        )
        boton_volver.click()
        time.sleep(0.3)

        print(f"✅ Registro {i + 1} procesado con éxito (Cédula: {cedula})")

        resultados.append({
            "CÉDULA": cedula,
            "CÓDIGO_PROGRAMA": codigo_programa,
            "FORMATO": periodo_buscar,
            "ESTADO": "Procesado con éxito"
        })

    except Exception as e:
        print(f"❌ Error en el registro {i + 1} (Cédula: {cedula}): {e}")
        # Si falla, intentar volver a la pantalla de búsqueda
        try:
            boton_volver = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'ui-button-text') and contains(text(), 'Volver')]"))
            )
            boton_volver.click()
            time.sleep(0.3)
        except:
            pass

    # Guardar como error
        resultados.append({
            "CÉDULA": cedula,
            "CÓDIGO_PROGRAMA": df['CODIGO_PROGRAMA'].iloc[i],
            "FORMATO": df['FORMATO'].iloc[i],
            "ESTADO": f"Error: {str(e).splitlines()[0]}"
        })

# Cerrar navegador al final
driver.quit()

# Guardar resultados
df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel("resultado_procesamiento.xlsx", index=False)
print("📄 Archivo 'resultado_procesamiento.xlsx' generado con los resultados.")